In [1]:
# after the trial in data_ingestion_trial get success, now it is time to create data for component folder

In [2]:
import os

In [3]:
pwd

'/Users/haojian/Desktop/work/coding + tech conversation/MLOps info/MLOps project/J-DeepLearning-image-app-with-mlflow-DVC/research'

In [4]:
# inorder to execute code we need to go the uplevel folder-J-DeepLearning-image-app-with-mlflow-DVC
os.chdir("../")

In [5]:
pwd

'/Users/haojian/Desktop/work/coding + tech conversation/MLOps info/MLOps project/J-DeepLearning-image-app-with-mlflow-DVC'

## 4. update entity:

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## 5.Update the configuration manager in src config[update the configure.py in src folde]

In [7]:
# now load the constant to the experimental file
from ImageClassifier.constants import *
# we also need to import the two function in utils folder to help us read yaml file
from ImageClassifier.utils.common import read_yaml, create_directories



In [8]:
class ConfigurationManager:
    # this init function is used to created the root directory artifacts for the entire project
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # this function is used to created the artifacts directories of data_ingestion component within the 
    # root directory of artifacts
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        # here config being assinged as the value of the key of data_ingestion in config.yaml file
        # it has four sub key: 
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## 6.Update the components: data ingesting.

In [11]:
import os
import zipfile
import gdown
from ImageClassifier import logger
from ImageClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
            '''
            Fetch data from the url
            '''

            try: 
                dataset_url = self.config.source_URL
                zip_download_dir = self.config.local_data_file
                os.makedirs("artifacts/data_ingestion", exist_ok=True)
                logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

                file_id = dataset_url.split("/")[-2]
                prefix = 'https://drive.google.com/uc?/export=download&id='
                gdown.download(prefix+file_id,zip_download_dir)

                logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

            except Exception as e:
                raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### 7.Update the pipeline: it is just a step wise chain calling a series method from the previous class we defined before, (similar as we built each substep in detail boefore, now we calling each substep one by one)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-05 21:14:18,553: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-05 21:14:18,582: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-05 21:14:18,588: INFO: common: created directory at: artifacts]
[2024-11-05 21:14:18,593: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-05 21:14:18,595: INFO: 596355673: Downloading data from https://drive.google.com/file/d/1fVh7BGAWDFAEYrFbI-pfsPzKJHlR6elo/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1fVh7BGAWDFAEYrFbI-pfsPzKJHlR6elo
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1fVh7BGAWDFAEYrFbI-pfsPzKJHlR6elo&confirm=t&uuid=8e34bd14-d10a-4efd-a2f3-b5685868bf34
To: /Users/haojian/Desktop/work/coding + tech conversation/MLOps info/MLOps project/J-DeepLearning-image-app-with-mlflow-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 49.1M/49.1M [00:04<00:00, 11.0MB/s]

[2024-11-05 21:14:27,952: INFO: 596355673: Downloaded data from https://drive.google.com/file/d/1fVh7BGAWDFAEYrFbI-pfsPzKJHlR6elo/view?usp=sharing into file artifacts/data_ingestion/data.zip]
